# Fake Rejection:

Check efficiency plots w.r.t the reference (noringer)

In [2]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency, restore_efficiencies
from Gaugi import mkdir_p, progressbar
from prettytable import PrettyTable
from tqdm import tqdm

import numpy as np
import pandas as pd
import collections
import os
import array
import root_numpy
import rootplotlib as rpl
from pprint import pprint
from copy import deepcopy
import gc
from ROOT import kBlack,kBlue,kRed,kAzure,kGreen,kMagenta,kCyan,kOrange,kGray,kYellow,kWhite,TColor,gStyle,TCanvas
import ROOT
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
GeV = 1000.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1) Read all files:

In [3]:
output_path = 'output/efficiencies'
mkdir_p(output_path)

In [8]:
# create my chain
chains = [
    Chain( "HLT_e17_lhvloose_nod0_noringer_L1EM15VHI", L1Seed = 'L1_EM15VHI' ),
    Chain( "HLT_e28_lhtight_nod0_noringer_ivarloose", L1Seed = 'L1_EM22VHI' ),
    Chain( "HLT_e60_lhmedium_nod0_noringer_L1EM24VHI", L1Seed = 'L1_EM24VHI' ),
    Chain( "HLT_e140_lhloose_nod0_noringer" , L1Seed = 'L1_EM24VHI' ),
    
    Chain( "HLT_e17_lhvloose_nod0_ringer_v17_L1EM15VHI"    , L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_v12_vloose',l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e28_lhtight_nod0_ringer_v17_ivarloose"     , L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_v12_tight' ,l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e60_lhmedium_nod0_ringer_v17_L1EM24VHI"    , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_medium',l2_column = 'ringer_v2_el_loose'),
    Chain( "HLT_e140_lhloose_nod0_ringer_v17"              , L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_v12_loose' ,l2_column = 'ringer_v2_el_loose'),
]

versions = ['v8', 
            #'v9', 
            #'v10', 
            #'v11',
            'v12',
            #'v13',
            #'v14',
            #'v15', 
            #'v16',
           ]

for v in versions:
    
    chains.extend( [

            Chain( "HLT_e17_lhvloose_nod0_ringer_%s_L1EM15VHI"%v, L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_%s_vloose'%v),
            Chain( "HLT_e28_lhtight_nod0_ringer_%s_ivarloose"%v, L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_%s_tight'%v ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_%s_L1EM24VHI"%v, L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_%s_medium'%v),
            Chain( "HLT_e140_lhloose_nod0_ringer_%s"%v , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_%s_loose'%v),

    ])


def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'trig_L2_cl_vloose_et12to22')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'trig_L2_cl_tight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'trig_L2_cl_medium_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'trig_L2_cl_loose_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])
OrderedDic

# 2017:

In [9]:
dpath   = '../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [10]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('v8/output/emulation/data17_table_v8.h5'),
                      #load_hdf('v9/output/emulation/data17_table_v9.h5'),
                      #load_hdf('v10/output/emulation/data17_table_v10.h5'),
                      #load_hdf('v11/output/emulation/data17_table_v11.h5'),
                      load_hdf('v12/output/emulation/data17_table_v12.h5'),
                      #load_hdf('v13/output/emulation/data17_table_v13.h5'),
                      #load_hdf('v14/output/emulation/data17_table_v14.h5'),
                      #load_hdf('v15/output/emulation/data17_table_v15.h5'),
                      #load_hdf('v16/output/emulation/data17_table_v16.h5'),
                      load_hdf('v2_el/output/emulation/data17_table_v2_el.h5'),
                     ), axis=1)

In [11]:
emulate(data_df)

Emulate...: 100%|██████████| 16/16 [05:42<00:00, 21.42s/it]


In [12]:
data_df.shape

(43311283, 182)

In [13]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(39861985, 182)

In [15]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer',
            'ringer_v8', 
            #'ringer_v9', 
            #'ringer_v10', 
            #'ringer_v11',
            'ringer_v12',
            #'ringer_v13',
            #'ringer_v14',
            #'ringer_v15', 
            #'ringer_v16',
            'ringer_v17',
           ]

for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version',
                      # FA
                      'L2CFA',
                      'L2CFA(<100)', 
                      'L2CFA(>100)',
                      'L2FA',
                      'L2FA(<100)',
                      'L2FA(>100)',
                      'HLT_FA', 
                      # PD
                      'L2CPD',
                      'L2PD' ,
                      'HLT_PD'] )
    
    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    is_bkg_low = (data_df['el_et']<100*GeV)&is_bkg
    is_bkg_high= (data_df['el_et']>=100*GeV)&is_bkg
    is_sgn_low = (data_df['el_et']<100*GeV)&is_sgn
    is_sgn_high= (data_df['el_et']>=100*GeV)&is_sgn
    
    total_sgn  = len(data_df.loc[is_sgn])
    total_bkg  = len(data_df.loc[is_bkg])


    print(trigger%'X')
    for version in tqdm( versions ):
        
            values = [version]
            
            #
            # FA
            #
            
            # L2Calo FA
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2Calo FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2Calo_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2Calo FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # L2 FA
            passed = len(data_df.loc[is_bkg&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2 FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2 FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # HLT FA
            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            fa = passed/total_bkg * 100           
            values.append( '%1.2f'%(fa) )
            
            
            #
            # PD
            #
            
            # L2Calo PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # L2 PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # HLT PD
            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            t.add_row(values)
    print(t)

            
            
            


e17_lhvloose_nod0_X_L1EM15VHI


100%|██████████| 4/4 [03:14<00:00, 48.66s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 35.83 |    13.46    |    43.98    | 35.77 |   13.44    |   43.90    |  0.29  | 94.48 | 94.34 | 92.17  |
| ringer_v8  |  2.24 |     2.58    |     2.12    |  2.16 |    2.56    |    2.02    |  0.17  | 94.66 | 94.51 | 92.34  |
| ringer_v12 |  2.14 |     2.55    |     1.98    |  2.07 |    2.53    |    1.90    |  0.16  | 94.65 | 94.51 | 92.34  |
| ringer_v17 |  2.14 |     2.55    |     1.98    |  0.59 |    1.29    |    0.34    |  0.09  | 94.65 | 94.43 | 92.28  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e28_lhtight_nod0_X_ivarloose


100%|██████████| 4/4 [02:39<00:00, 39.91s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 26.47 |     7.80    |    30.57    | 26.41 |    7.79    |   30.50    |  0.00  | 93.98 | 93.87 | 86.84  |
| ringer_v8  |  0.84 |     1.08    |     0.78    |  0.79 |    1.07    |    0.73    |  0.00  | 94.22 | 94.11 | 87.01  |
| ringer_v12 |  0.82 |     1.12    |     0.76    |  0.78 |    1.11    |    0.70    |  0.00  | 94.22 | 94.11 | 87.01  |
| ringer_v17 |  0.82 |     1.12    |     0.76    |  0.35 |    0.72    |    0.27    |  0.00  | 94.22 | 94.08 | 87.00  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e60_lhmedium_nod0_X_L1EM24VHI


100%|██████████| 4/4 [01:03<00:00, 15.79s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 34.45 |    17.60    |    35.03    | 34.37 |   17.59    |   34.95    |  0.01  | 97.31 | 97.22 | 91.87  |
| ringer_v8  |  0.83 |     0.38    |     0.85    |  0.78 |    0.37    |    0.80    |  0.00  | 97.36 | 97.27 | 91.90  |
| ringer_v12 |  0.81 |     0.44    |     0.83    |  0.76 |    0.44    |    0.77    |  0.00  | 97.35 | 97.26 | 91.90  |
| ringer_v17 |  0.81 |     0.44    |     0.83    |  0.28 |    0.25    |    0.28    |  0.00  | 97.35 | 97.18 | 91.85  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e140_lhloose_nod0_X


100%|██████████| 4/4 [00:46<00:00, 11.74s/it]

+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 94.57 |     0.00    |    94.57    | 69.06 |    0.00    |   69.06    |  0.07  | 98.19 | 98.13 | 95.15  |
| ringer_v8  |  2.06 |     0.00    |     2.06    |  1.96 |    0.00    |    1.96    |  0.05  | 98.16 | 98.09 | 95.14  |
| ringer_v12 |  1.92 |     0.00    |     1.92    |  1.83 |    0.00    |    1.83    |  0.05  | 98.15 | 98.08 | 95.13  |
| ringer_v17 |  1.92 |     0.00    |     1.92    |  0.36 |    0.00    |    0.36    |  0.02  | 98.15 | 97.61 | 94.78  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+


# 2018:

In [17]:
dpath   = '../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [18]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('v8/output/emulation/data18_table_v8.h5'),
                      #load_hdf('v9/output/emulation/data18_table_v9.h5'),
                      #load_hdf('v10/output/emulation/data18_table_v10.h5'),
                      #load_hdf('v11/output/emulation/data18_table_v11.h5'),
                      load_hdf('v12/output/emulation/data18_table_v12.h5'),
                      #load_hdf('v13/output/emulation/data18_table_v13.h5'),
                      #load_hdf('v14/output/emulation/data18_table_v14.h5'),
                      #load_hdf('v15/output/emulation/data18_table_v15.h5'),
                      #load_hdf('v16/output/emulation/data18_table_v16.h5'),
                      load_hdf('v2_el/output/emulation/data18_table_v2_el.h5'),
                     ), axis=1)

In [19]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(38543015, 102)

In [20]:
emulate(data_df)

Emulate...: 100%|██████████| 16/16 [05:01<00:00, 18.83s/it]


In [21]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer',
            'ringer_v8', 
            #'ringer_v9', 
            #'ringer_v10', 
            #'ringer_v11',
            'ringer_v12',
            #'ringer_v13',
            #'ringer_v14',
            #'ringer_v15', 
            #'ringer_v16',
            'ringer_v17']

for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version',
                      # FA
                      'L2CFA',
                      'L2CFA(<100)', 
                      'L2CFA(>100)',
                      'L2FA',
                      'L2FA(<100)',
                      'L2FA(>100)',
                      'HLT_FA', 
                      # PD
                      'L2CPD',
                      'L2PD' ,
                      'HLT_PD'] )
    
    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    is_bkg_low = (data_df['el_et']<100*GeV)&is_bkg
    is_bkg_high= (data_df['el_et']>=100*GeV)&is_bkg
    is_sgn_low = (data_df['el_et']<100*GeV)&is_sgn
    is_sgn_high= (data_df['el_et']>=100*GeV)&is_sgn
    
    total_sgn  = len(data_df.loc[is_sgn])
    total_bkg  = len(data_df.loc[is_bkg])


    print(trigger%'X')
    for version in tqdm( versions ):
        
            values = [version]
            
            #
            # FA
            #
            
            # L2Calo FA
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2Calo FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2Calo_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2Calo FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # L2 FA
            passed = len(data_df.loc[is_bkg&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )
            
            # L2 FA < 100
            total  = len(data_df.loc[is_bkg_low])
            if total > 0:
                passed = len(data_df.loc[is_bkg_low&(data_df['L2_'+trigger%version]==True)])
                fa = passed/total * 100           
                values.append( '%1.2f'%(fa) )
            else:
                values.append( '%1.2f'%(0) )
               
            # L2 FA > 100
            passed = len(data_df.loc[is_bkg_high&(data_df['L2_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg_high])
            fa = passed/total * 100           
            values.append( '%1.2f'%(fa) )

            # HLT FA
            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            fa = passed/total_bkg * 100           
            values.append( '%1.2f'%(fa) )
            
            
            #
            # PD
            #
            
            # L2Calo PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # L2 PD
            passed = len(data_df.loc[ is_sgn & (data_df['L2_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            
            # HLT PD
            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            det = passed/total_sgn * 100
            values.append( '%1.2f'%(det) )
            
            t.add_row(values)
    print(t)

            
            

e17_lhvloose_nod0_X_L1EM15VHI


100%|██████████| 4/4 [03:13<00:00, 48.43s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 36.80 |    13.77    |    44.41    | 36.74 |   13.76    |   44.34    |  1.02  | 94.45 | 94.32 | 92.34  |
| ringer_v8  |  2.71 |     3.03    |     2.61    |  2.63 |    3.01    |    2.50    |  0.69  | 94.64 | 94.50 | 92.52  |
| ringer_v12 |  2.61 |     3.02    |     2.47    |  2.54 |    3.00    |    2.38    |  0.68  | 94.63 | 94.50 | 92.52  |
| ringer_v17 |  2.61 |     3.02    |     2.47    |  0.95 |    1.73    |    0.70    |  0.46  | 94.63 | 94.42 | 92.47  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e28_lhtight_nod0_X_ivarloose


100%|██████████| 4/4 [02:30<00:00, 37.66s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 27.30 |     8.31    |    31.07    | 27.24 |    8.30    |   31.00    |  0.06  | 93.93 | 93.84 | 87.38  |
| ringer_v8  |  1.20 |     1.54    |     1.13    |  1.15 |    1.53    |    1.08    |  0.05  | 94.19 | 94.09 | 87.58  |
| ringer_v12 |  1.18 |     1.58    |     1.10    |  1.13 |    1.57    |    1.04    |  0.05  | 94.20 | 94.10 | 87.60  |
| ringer_v17 |  1.18 |     1.58    |     1.10    |  0.68 |    1.17    |    0.59    |  0.04  | 94.20 | 94.07 | 87.58  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e60_lhmedium_nod0_X_L1EM24VHI


100%|██████████| 4/4 [00:54<00:00, 13.65s/it]


+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 34.98 |    17.88    |    35.51    | 34.91 |   17.88    |   35.43    |  0.08  | 97.31 | 97.22 | 92.14  |
| ringer_v8  |  1.19 |     0.58    |     1.21    |  1.14 |    0.57    |    1.16    |  0.06  | 97.35 | 97.26 | 92.19  |
| ringer_v12 |  1.16 |     0.63    |     1.18    |  1.11 |    0.63    |    1.12    |  0.06  | 97.37 | 97.28 | 92.21  |
| ringer_v17 |  1.16 |     0.63    |     1.18    |  0.60 |    0.46    |    0.60    |  0.05  | 97.37 | 97.20 | 92.16  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
e140_lhloose_nod0_X


100%|██████████| 4/4 [00:40<00:00, 10.14s/it]

+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  version   | L2CFA | L2CFA(<100) | L2CFA(>100) |  L2FA | L2FA(<100) | L2FA(>100) | HLT_FA | L2CPD |  L2PD | HLT_PD |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
|  noringer  | 94.64 |     0.00    |    94.64    | 64.74 |    0.00    |   64.74    |  0.43  | 98.20 | 98.13 | 95.45  |
| ringer_v8  |  2.62 |     0.00    |     2.62    |  2.52 |    0.00    |    2.52    |  0.36  | 98.23 | 98.16 | 95.49  |
| ringer_v12 |  2.48 |     0.00    |     2.48    |  2.39 |    0.00    |    2.39    |  0.36  | 98.21 | 98.14 | 95.47  |
| ringer_v17 |  2.48 |     0.00    |     2.48    |  0.77 |    0.00    |    0.77    |  0.28  | 98.21 | 97.72 | 95.17  |
+------------+-------+-------------+-------------+-------+------------+------------+--------+-------+-------+--------+
